In [53]:
#Importing libraries
import numpy as np
import math as m
import pandas as pd
import matplotlib.pyplot as plt

In [54]:

#Import data from roads
InitialRoadData = pd.read_csv('..\RMMS\CSV\_allRoads.csv')
RoadData = InitialRoadData


In [55]:
#Deleting inconsistent coordinates 
#Finding quantities out of tolerance. This tolerance may be improved by checking average differences!!
Tolerance = 0.02
RoadData['UpperErrorLat']=abs(RoadData['Latitude'].shift(-1)-RoadData['Latitude'])>Tolerance
RoadData['BelowErrorLat']=abs(RoadData['Latitude'].shift()-RoadData['Latitude'])>Tolerance
RoadData['UpperErrorLon']=abs(RoadData['Longitude'].shift(-1)-RoadData['Longitude'])>Tolerance
RoadData['BelowErrorLon']=abs(RoadData['Longitude'].shift()-RoadData['Longitude'])>Tolerance
#Checking Road ID
RoadData['RoadIDCheckAbove']=RoadData['Road Id']==RoadData['Road Id'].shift(-1)
RoadData['RoadIDCheckBelow']=RoadData['Road Id']==RoadData['Road Id'].shift()
#Labeling Errors
RoadData['Error']=((RoadData['RoadIDCheckAbove'] & RoadData['RoadIDCheckBelow'] & ((RoadData['UpperErrorLat'] & RoadData['BelowErrorLat'])|(RoadData['UpperErrorLon'] & RoadData['BelowErrorLon'])))|
                   (RoadData['RoadIDCheckAbove'] & -RoadData['RoadIDCheckBelow'] & ((RoadData['UpperErrorLat'] &  -RoadData['UpperErrorLat'].shift(-1))|(RoadData['UpperErrorLon'] &  -RoadData['UpperErrorLon'].shift(-1))))|
                   (-RoadData['RoadIDCheckAbove'] & RoadData['RoadIDCheckBelow'] & ((RoadData['BelowErrorLat'] &  -RoadData['BelowErrorLat'].shift())|(RoadData['BelowErrorLon'] &  -RoadData['BelowErrorLon'].shift())))
                  )


In [56]:
#Erasing Wrong values
RoadData.loc[RoadData.Error == True, 'Latitude'] = np.NaN
RoadData.loc[RoadData.Error == True, 'Longitude'] = np.NaN      


In [57]:
#Interpolating 
SmoothRoadData = RoadData.interpolate()
#Formatting
del SmoothRoadData['UpperErrorLat']
del SmoothRoadData['UpperErrorLon']
del SmoothRoadData['BelowErrorLat']
del SmoothRoadData['BelowErrorLon']
del SmoothRoadData['RoadIDCheckAbove']
del SmoothRoadData['RoadIDCheckBelow']
del SmoothRoadData['Error']
SmoothRoadData.columns=SmoothRoadData.columns.str.replace('LRP Number','IDnumber')
SmoothRoadData.columns=SmoothRoadData.columns.str.replace('Road Id','RoadNo')



In [58]:
#Load Bridge Data
InitialBridgeData = pd.read_csv('..\BMMS\CSV\_allBridges.csv')
#This texts deletes NaN values
BridgeData = InitialBridgeData
BridgeData.columns=BridgeData.columns.str.replace('BridgeNo','IDnumber')

In [59]:
#Merge Data Frames
TotalData = pd.merge(BridgeData, SmoothRoadData, on=['RoadNo','IDnumber'], how='outer')
TotalData = TotalData.fillna(-1)


In [60]:
#Detecting Errors
#Tolerance of 100m
Tolerance=0.00008
TotalData['LatitudeNotMatching']=abs(TotalData.Latitude_x-TotalData.Latitude_y)>Tolerance  
TotalData['LongitudeNotMatching']=abs(TotalData.Longitude_x-TotalData.Longitude_y)>Tolerance
TotalData['NoDataInRoadFile']=TotalData.Latitude_y==-1
TotalData['NoDataInBridgeFile']=TotalData.Latitude_x==-1
TotalData.loc[TotalData['LatitudeNotMatching']&TotalData['LongitudeNotMatching']&-TotalData['NoDataInRoadFile']&-TotalData['NoDataInBridgeFile'],'TypeOfError']='LatAndLongNotMatching'
TotalData.loc[TotalData['LatitudeNotMatching']&-TotalData['LongitudeNotMatching']&-TotalData['NoDataInRoadFile']&-TotalData['NoDataInBridgeFile'],'TypeOfError']='LatNotMatching'
TotalData.loc[-TotalData['LatitudeNotMatching']&TotalData['LongitudeNotMatching']&-TotalData['NoDataInRoadFile']&-TotalData['NoDataInBridgeFile'],'TypeOfError']='LongNotMatching'
TotalData.loc[TotalData['NoDataInRoadFile']&TotalData['NoDataInBridgeFile'],'TypeOfError']='NoDataInRoadAndBridgeFile'
TotalData.loc[TotalData['NoDataInRoadFile']&-TotalData['NoDataInBridgeFile'],'TypeOfError']='NoDataInRoadFile'
TotalData.loc[-TotalData['NoDataInRoadFile']&TotalData['NoDataInBridgeFile'],'TypeOfError']='NoDataInBridgeFile'
TotalData.loc[-TotalData['LatitudeNotMatching']&-TotalData['LongitudeNotMatching']&-TotalData['NoDataInRoadFile']&-TotalData['NoDataInBridgeFile'],'TypeOfError']='CorrectData'




In [61]:
#Format Final File
FinalData=TotalData
del FinalData['LatitudeNotMatching']
del FinalData['LongitudeNotMatching']
del FinalData['NoDataInRoadFile']
del FinalData['NoDataInBridgeFile']
FinalData.columns=FinalData.columns.str.replace('Latitude_x','Latitude_B')
FinalData.columns=FinalData.columns.str.replace('Latitude_y','Latitude_R')
FinalData.columns=FinalData.columns.str.replace('Longitude_x','Longitude_B')
FinalData.columns=FinalData.columns.str.replace('Longitude_y','Longitude_R')


In [62]:
FinalData[FinalData['TypeOfError']=='NoDataInRoadAndBridgeFile']

Number               Structure Name Condition of Bridge  TotalWidth  \
11        12.0           Madanpur Bridge(R)                   A        -1.0   
18        19.0           Sonakhal Bridge(R)                   B        -1.0   
24        25.0          Marikhali Bridge(R)                   A        -1.0   
27        28.0       Ashariarchar Bridge(R)                   B        -1.0   
29        30.0     Asharirchar-2 Bridge.(R)                   A        -1.0   
41        42.0        Bhaber char Bridge(L)                   A        -1.0   
43        44.0            Baushia Bridge(L)                   A        -1.0   
51        52.0   Baushia bazar Bridge ( R )                   A        -1.0   
131      132.0             Rampur Bridge(R)                   A        -1.0   
1470    1466.0            Ram Bhanga Bridge                   A        -1.0   
2392    2384.0                  Amla Bridge                   B        -1.0   
3189    3179.0                           -1                   A        -1.0   
3481    3470.0              Basagari Bridge                   B        -1.0   
4002    3989.0               Gozaria Bridge                   A        -1.0   
4501    4485.0                 Godir Bridge                   C        -1.0   
4555    4539.0              Kharabor Bridge                   B        -1.0   
5500    5482.0       Khorshed Market Bridge                   B        -1.0   
6105    6084.0           Talim Nagar Bridge                   B        -1.0   
6312    6291.0  Mir Mosharraf Hosain Bridge                   D        -1.0   
7122    7095.0      Amragachia Bazar Bridge                   A        -1.0   
7570    7543.0            Kamar Khal Bridge                   A        -1.0   
9448    9418.0          College Road Bridge                   B        -1.0   
10720  10686.0            Halder Par Bridge                   B        -1.0   
10934  10900.0                           -1                   B        -1.0   
10938  10904.0             Teli Khal Bridge                   B        -1.0   
11050  11016.0              Badalpur Bridge                   A        -1.0   
11392  11358.0             Dewanpara Bridge                   C        -1.0   
13789  13751.0      Baliabari bailey Bridge                   C        -1.0   
13850  13812.0    Chaahkair Dhauhata Bridge                   C        -1.0   
13975  13937.0            Khonagatii Bridge                   C        -1.0   
14129  14091.0          Baliaghata-3 Bridge                   B        -1.0   
14500  14460.0   Naodanga Pooler Par Bridge                   C        -1.0   
14667  14627.0                Gathia Bridge                   D        -1.0   
15803  15759.0          Ibrahimnagar Bridge                   A        -1.0   
15806  15762.0             Tatipukur Bridge                   C        -1.0   
16389  16342.0                Siddhi Bridge                   C        -1.0   
17103  17056.0              Nazirpur Bridge                   A        -1.0   
17307  17260.0           Sirajdikhan Bridge                   B        -1.0   
17557  17510.0                Soular Bridge                   A        -1.0   
17669  17622.0            Gobindapur bridge                   B        -1.0   
17707  17660.0            Baishroshi Bridge                   C        -1.0   
18085  18037.0     Patharghata Steel Bridge                   D        -1.0   
18296  18241.0             Charkhali bridge                   B        -1.0   
18372  18317.0                           -1                   C        -1.0   

       TotalLength  ConstructionYear  NumberOfSpan        Zone         Circle  \
11            -1.0              -1.0          -1.0       Dhaka          Dhaka   
18            -1.0              -1.0          -1.0       Dhaka          Dhaka   
24            -1.0              -1.0          -1.0       Dhaka          Dhaka   
27            -1.0              -1.0          -1.0       Dhaka          Dhaka   
29            -1.0              -1.0          -

In [63]:
FinalData.to_csv('TinderFile.csv')